In [1]:
# --- 0) bootstrap ---
import os, sys, json, asyncio, time, textwrap
from pathlib import Path

PROJECT_ROOT = Path.cwd()
while not (PROJECT_ROOT / "pyproject.toml").exists() and PROJECT_ROOT != PROJECT_ROOT.parent:
    PROJECT_ROOT = PROJECT_ROOT.parent
os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
print("Project root:", PROJECT_ROOT)

DOC_ID = "NFS_2019"

Project root: d:\IIT BBS\Job Resources\Business Optima\new-pdf-agent


In [2]:
from packages.chat.router import run_turn
from packages.chat.memory import SessionStore

sess = SessionStore(Path("data/sessions"), DOC_ID).load_or_create()
sid = sess.session_id

async def run_and_collect(q):
    buf, types = "", []
    async for ev in run_turn(DOC_ID, sid, q):
        t = ev.get("type"); types.append(t)
        if t == "final_token":
            tok = ev.get("data",""); buf += tok
    print("TYPES:", [t for t in types if t!="final_token"])
    print("— FINAL —\n", buf or "[no text]")

print(">>> OOS 1")
await run_and_collect("Who won the Premier League in 2021?")

print("\n>>> IN-SCOPE 1")
await run_and_collect("Define 'conversion factor' used in this document.")

print("\n>>> IN-SCOPE 2")
await run_and_collect("What are Section 3 fees?")

>>> OOS 1


d:\IIT BBS\Job Resources\Business Optima\new-pdf-agent\packages\chat\models.py:164: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  return ChatOllama(


TYPES: ['guard', 'guard_override', 'split', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_final', 'final']
— FINAL —
 I'm ready to assist. What's the question?

>>> IN-SCOPE 1
TYPES: ['guard', 'split', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token', 'core_token'

In [2]:
from packages.chat.router import mount_chat
mount = mount_chat(DOC_ID)
print("Sessions dir:", mount.sessions_dir)
print("Adapter used:", mount.profile.adapter_path)
print("Collection:  ", mount.profile.collection)

Sessions dir: data\sessions
Adapter used: data\models\NFS_2019\20bb948f\adapter
Collection:   NFS_2019


d:\IIT BBS\Job Resources\Business Optima\new-pdf-agent\packages\chat\models.py:164: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  return ChatOllama(


In [3]:
import requests, yaml

def check_roles_health(yaml_path="configs/providers.yaml", roles=("intro","splitter","core","output")):
    cfg = yaml.safe_load(open(yaml_path, "r", encoding="utf-8"))
    failed = []
    for role in roles:
        rcfg = cfg["chat"]["models"][role]
        if rcfg.get("provider") != "ollama":
            print(f"{role:9s} (non-ollama provider) -> skip")
            continue
        url = rcfg["base_url"]
        try:
            r = requests.get(f"{url}/api/tags", timeout=10)
            print(f"{role:9s} {url} -> {r.status_code}")
            if r.status_code != 200:
                failed.append((role, url, f"HTTP {r.status_code}"))
        except Exception as e:
            print(f"{role:9s} {url} -> ERROR: {type(e).__name__}: {e}")
            failed.append((role, url, str(e)))
    return failed

fails = check_roles_health()
if fails:
    raise SystemExit("Ollama health check failed:\n" + "\n".join(f"{r} @ {u}: {e}" for r,u,e in fails))

intro     http://127.0.0.1:11434 -> 200
splitter  http://127.0.0.1:11434 -> 200
core      http://127.0.0.1:11434 -> 200
output    http://127.0.0.1:11434 -> 200


In [4]:
from packages.chat.router import run_turn

async def run_and_collect(doc_id: str, session_id: str, user_query: str, print_tokens: bool = True):
    stream_buf = []
    events = []
    async for ev in run_turn(doc_id, session_id=session_id, user_query=user_query):
        events.append(ev)
        t = ev.get("type")
        if t == "final_token" and print_tokens:
            token = ev.get("data", "")
            stream_buf.append(token)
            # lightweight preview
            if token.strip():
                print(token, end="", flush=True)
    if print_tokens:
        print("\n\n— FINAL —")
        print("".join(stream_buf))
    return events

# fresh session for 7F tests
from packages.chat.memory import SessionStore
store = SessionStore(mount.sessions_dir, DOC_ID).load_or_create()
sid = store.session_id
print("Session file:", store.path)

Session file: data\sessions\NFS_2019_83837ce1d49b.json


In [ ]:
print(">>> OOS")
evs_oos = await run_and_collect(DOC_ID, sid, "Who won the Premier League in 2021?")
# Inspect non-token events you might want to log/trace
[e["type"] for e in evs_oos if e["type"] != "final_token"]

>>> OOS
This request is out of scope for the selected PDF. (reason: Premier League winner)

— FINAL —
This request is out of scope for the selected PDF. (reason: Premier League winner)


['guard', 'final']

In [6]:
print(">>> In-scope")
evs_in = await run_and_collect(DOC_ID, sid, "Define 'conversion factor' used in this document.")
# Quick peek at event types
[e["type"] for e in evs_in if e["type"] != "final_token"]

>>> In-scope
This request is out of scope for the selected PDF.

— FINAL —
This request is out of scope for the selected PDF.


['guard',
 'split',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'core_token',
 'cor

In [8]:
from packages.chat.memory import SessionStore
store2 = SessionStore(mount.sessions_dir, DOC_ID, session_id=sid).load_or_create()
print("Summary:\n", store2.state.summary)
print("\nTail turns:", [f"[{t.role}] {t.content[:80]}" for t in store2.last_n(6)])

Summary:
 There was no prior summary. The user asked about the winner of the Premier League in 2021, and I replied that I couldn't provide an accurate answer as it's out of scope. I suggested checking the official Premier League website or sports news websites for more information.

Tail turns: ['[user] Who won the Premier League in 2021?', "[assistant] Unfortunately, I don't have enough information to provide an accurate response. ", "[user] Define 'conversion factor' used in this document.", "[assistant] Unfortunately, I don't have enough information to provide the winner of the Prem"]


In [9]:
# Collect final text from prior run (evs_in) if needed:
final_text = "".join(ev["data"] for ev in evs_in if ev["type"] == "final_token").strip()
Path("data/tmp").mkdir(parents=True, exist_ok=True)
Path("data/tmp/last_final.txt").write_text(final_text, encoding="utf-8")
print("Saved:", "data/tmp/last_final.txt")

Saved: data/tmp/last_final.txt
